<a href="https://colab.research.google.com/github/HuangJin-De/am_NH/blob/main/discussion/20231115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! rm -rf am_NH
! git clone https://github.com/HuangJin-De/am_NH.git

Cloning into 'am_NH'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 216 (delta 94), reused 128 (delta 41), pack-reused 0
Receiving objects: 100% (216/216), 7.58 MiB | 18.92 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [7]:
import numpy as np
import matplotlib.pyplot as plt

path='am_NH'
filen=path+'/train_data/zm_21days_ori_data.dat'

data=np.fromfile(filen,dtype=np.float32)
data=data.reshape(-1,84)

z1=np.copy(data[:,0:21])
z2=np.copy(data[:,21:42])
m1=np.copy(data[:,42:63])
m2=np.copy(data[:,63:84])

print(z1.shape,z2.shape,m1.shape,m2.shape)
print(z1[0,:])
print(m1[0,:])

(4242, 21) (4242, 21) (4242, 21) (4242, 21)
[ 27.807383    29.223433    30.451048    34.08071     28.950182
  24.049631    23.813913    25.54383     22.154768    16.67688
  12.014231     6.624656    -0.60133517  -4.5212555   -7.4034467
  -6.4296618   -3.5772855   -6.931867   -10.304456    -4.7410808
   1.5217469 ]
[11.36987     2.6580744   5.997234    5.95777    -3.3631287   2.5488837
  6.3826714   2.6302502  -2.3663964  -3.7961032  -0.500228   -5.9676085
 -7.0242863  -4.391776   -3.4341097   0.62892973 -0.78749216 -6.9614167
  1.1405915   7.594978    3.602327  ]


estimations of b11, b12, b21, and b22.

In [12]:
from sklearn.linear_model import LinearRegression

def cal_reg_coef(x,y):
  model=LinearRegression()
  model.fit(x,y)
  return model.coef_


lag_z1z1=[]
lag_z1z2=[]
lag_z2z1=[]
lag_z2z2=[]
lag_m1z1=[]
lag_m1z2=[]
lag_m2z1=[]
lag_m2z2=[]

n=0
for i in np.arange(8,21):
  x=np.copy(z1[:,n]).reshape(-1,1)
  y=np.copy(z1[:,i]).reshape(-1,1)
  lag_z1z1.append(cal_reg_coef(x,y))

  x=np.copy(z1[:,n]).reshape(-1,1)
  y=np.copy(z2[:,i]).reshape(-1,1)
  lag_z2z1.append(cal_reg_coef(x,y))

  x=np.copy(z2[:,n]).reshape(-1,1)
  y=np.copy(z1[:,i]).reshape(-1,1)
  lag_z1z2.append(cal_reg_coef(x,y))

  x=np.copy(z2[:,n]).reshape(-1,1)
  y=np.copy(z2[:,i]).reshape(-1,1)
  lag_z2z2.append(cal_reg_coef(x,y))

  x=np.copy(z1[:,n]).reshape(-1,1)
  y=np.copy(m1[:,i]).reshape(-1,1)
  lag_m1z1.append(cal_reg_coef(x,y))

  x=np.copy(z2[:,n]).reshape(-1,1)
  y=np.copy(m1[:,i]).reshape(-1,1)
  lag_m1z2.append(cal_reg_coef(x,y))

  x=np.copy(z1[:,n]).reshape(-1,1)
  y=np.copy(m2[:,i]).reshape(-1,1)
  lag_m2z1.append(cal_reg_coef(x,y))

  x=np.copy(z2[:,n]).reshape(-1,1)
  y=np.copy(m2[:,i]).reshape(-1,1)
  lag_m2z2.append(cal_reg_coef(x,y))


#print(np.mean(lag_m1z1))
#print(np.mean(lag_m1z2))

a1=np.matrix([[np.mean(lag_z1z1), np.mean(lag_z2z1)],
 [np.mean(lag_z1z2), np.mean(lag_z2z2)]])
b1=np.matrix([[np.mean(lag_m1z1)], [np.mean(lag_m1z2)]])

x1=np.matmul(np.linalg.inv(a1),b1)

print(x1)

a2=np.matrix([[np.mean(lag_z2z1), np.mean(lag_z1z1)],
 [np.mean(lag_z2z2), np.mean(lag_z1z2)]])
b2=np.matrix([[np.mean(lag_m2z1)], [np.mean(lag_m2z2)]])

x2=np.matmul(np.linalg.inv(a2),b2)

print(x2)


[[0.05786296]
 [0.02330417]]
[[ 0.09109624]
 [-0.05169249]]
